In [ ]:
data_in = pd.read_csv("_data/_TESS_lightcurves_outliercleaned/ACV/92705248_sector01_1_1_cleaned.lc",sep=',',
names = ['time', 'mag', 'magerr'])
tess_lc = data_in.to_records(index=False)

In [ ]:
import astropy.stats
from astropy.timeseries import LombScargle


In [ ]:
# Ivezic v2, Figure 10.15, edits by GTR (see UW AST324) and SRT
# License: BSD
#   The figure produced by this code is published in the textbook
#   "Statistics, Data Mining, and Machine Learning in Astronomy" (2013)
#   For more information, see http://astroML.github.com
#   To report a bug or issue, use the following forum:
#    https://groups.google.com/forum/#!forum/astroml-general


from astroML.time_series import\
    lomb_scargle, lomb_scargle_BIC, lomb_scargle_bootstrap

#Need to switch to this in the future
from astropy.timeseries import LombScargle


#import astropy.stats.LombScargle()
#----------------------------------------------------------------------
# This function adjusts matplotlib settings for a uniform feel in the textbook.
# Note that with usetex=True, fonts are rendered with LaTeX.  This may
# result in an error if LaTeX is not installed on your system.  In that case,
# you can set usetex to False.
if "setup_text_plots" not in globals():
    from astroML.plotting import setup_text_plots
setup_text_plots(fontsize=12, usetex=False)
from gatspy.periodic import LombScargle
#------------------------------------------------------------
# Generate Data
np.random.seed(0)
N = 50 # was originally 100
#P = 0.5 # was originally 0.5 # 50

#t = np.random.randint(100, size=N) + 0.3 + 0.4 * np.random.random(N)
#y = 10 + np.sin(2 * np.pi * t / P)


a = tess_lc['time'][0:60]
b = tess_lc['mag'][0:60]
t = a
y = b

dy = tess_lc['magerr'][0:60]
#t = data['time']
#y = data['mag']

#dy = 0.1 + 0.1 * np.random.random(N)
y_obs = np.random.normal(y, dy)

#------------------------------------------------------------
# Compute periodogram
period = 10 ** np.linspace(-1, 0, 10000)
omega = 2 * np.pi / period
PS = lomb_scargle(t, y_obs, dy, 
                  omega, generalized=True)



#------------------------------------------------------------
# Get significance via bootstrap
#D = lomb_scargle_bootstrap(t, y_obs, dy, omega, generalized=True, 
#                           N_bootstraps=1000, random_state=0)
#Need to switch to this in the future
D = LombScargle(t, y, dy)
print(D)
sig1, sig5 = np.percentile(D, [99, 99.5])

#------------------------------------------------------------
# Plot the results
fig = plt.figure(figsize=(10, 5))
fig.subplots_adjust(left=0.1, right=0.9, hspace=0.35)

# First panel: the data
ax = fig.add_subplot(211)
ax.errorbar(t, y_obs, dy, fmt='.k', lw=1, ecolor='gray')
ax.set_xlabel('time (days)')
ax.set_ylabel('flux')
ax.set_xlim(np.min(t),np.max(t))

# Second panel: the periodogram & significance levels
ax1 = fig.add_subplot(212, xscale='log')
ax1.plot(period, PS, '-', 
         c='black', lw=1, zorder=1)
ax1.plot([period[0], period[-1]], 
         [sig1, sig1], ':', c='black')
ax1.plot([period[0], period[-1]], 
         [sig5, sig5], ':', c='black')


ax1.set_xlim(period[0], period[-1])
ax1.set_ylim(-0.05, 0.85)

ax1.set_xlabel(r'period (days)')
ax1.set_ylabel('power')

# Twin axis: label BIC on the right side
ax2 = ax1.twinx()
ax2.set_ylim(tuple(lomb_scargle_BIC(ax1.get_ylim(), y_obs, dy)))
ax2.set_ylabel(r'$\Delta BIC$')

ax1.xaxis.set_major_formatter(plt.FormatStrFormatter('%.1f'))
ax1.xaxis.set_minor_formatter(plt.FormatStrFormatter('%.1f'))
ax1.xaxis.set_major_locator(plt.LogLocator(10))
ax1.xaxis.set_major_formatter(plt.FormatStrFormatter('%.3g'))

plt.show()